In [1]:
# import numpy as np
import cvxpy as cp
import networkx as nx
import autograd.numpy as np
from autograd import grad

import pymanopt
import pymanopt.manifolds
import pymanopt.solvers

from numpy.random import default_rng
from complex_elliptope import ComplexElliptope

from opt_utils import decompose_psd, normalize_rows, complex_hyperplane_rounding

In [2]:
def get_rand_complex(m, n, max_val=5):
    return np.round(np.random.rand(m, n) * max_val) + np.round(np.random.rand(m, n) * max_val) * 1j

# Phase recovery

In [3]:
n = 50  # number of observations
p = 20  # dimension of x
max_val = 1e2

In [4]:
rng = default_rng()
A = rng.random((n,p)) * max_val + rng.random((n,p)) * max_val * 1j
assert np.linalg.matrix_rank(A, tol=1e-6) >= p  # A must be injective; if the rows of A are linearly independent, AA+ = I
b = rng.random(n) * max_val
M = np.diag(b) @ (np.identity(n) - A @ np.linalg.pinv(A)) @ np.diag(b)

\begin{equation*}
\begin{aligned}
\min \quad & u^{*}Mu \\
\textrm{s.t.} \quad & |u_{i}| = 1, \qquad i=1,...,n
\end{aligned}
\end{equation*}

In [15]:
# u = cp.Variable(n, complex=True)
# qp_constraints = [ cp.square(cp.real(u[i])) + cp.square(cp.imag(u[i])) == 1 for i in range(n) ]
# prob = cp.Problem(cp.Minimize(cp.quad_form(u, M)), qp_constraints)
# prob.solve()

\begin{equation*}
\begin{aligned}
\min \quad & M \cdot U \\
\textrm{s.t.} \quad & U_{ii} = 1, \qquad i=1,...,n \\
& U \succeq 0
\end{aligned}
\end{equation*}

### solve SDP with cvxpy

In [9]:
U = cp.Variable((n,n), hermitian=True)
# U = cp.Variable((n,n), symmetric=True)  # for testing

In [10]:
sdp_constraints = [U >> 0]
sdp_constraints += [U[i][i] == 1 for i in range(n)]

In [11]:
prob = cp.Problem(cp.Maximize(cp.real(cp.trace(M @ U))), sdp_constraints)
prob.solve()

print("The optimal value is", prob.value)
print("A solution U is", U.value)

The optimal value is 227998.78899964158
A solution U is [[ 9.99999997e-01+0.j          9.43357384e-02+0.27514311j
  -8.32239114e-02+0.08957215j ...  7.56581847e-02+0.15751748j
   1.64663577e-01-0.32299026j -3.96778018e-03-0.00885349j]
 [ 9.43357384e-02-0.27514311j  9.99999997e-01+0.j
  -1.45497364e-01+0.05757336j ...  2.54574326e-01-0.1440652j
  -1.08787821e-01-0.25907406j -1.28752937e-02-0.00554008j]
 [-8.32239114e-02-0.08957215j -1.45497364e-01-0.05757336j
   9.99999996e-01+0.j         ... -2.16986718e-02-0.04728792j
  -1.44870767e-01-0.07348178j  8.05666979e-04+0.00418659j]
 ...
 [ 7.56581847e-02-0.15751748j  2.54574326e-01+0.1440652j
  -2.16986718e-02+0.04728792j ...  9.99999997e-01+0.j
  -4.23641823e-02-0.01645329j  1.11103137e-03-0.00177536j]
 [ 1.64663577e-01+0.32299026j -1.08787821e-01+0.25907406j
  -1.44870767e-01+0.07348178j ... -4.23641823e-02+0.01645329j
   9.99999997e-01+0.j          2.47040835e-03+0.00345465j]
 [-3.96778018e-03+0.00885349j -1.28752937e-02+0.00554008j
   8

In [12]:
opt_rank = np.linalg.matrix_rank(U.value, 1e-9)
print(opt_rank)

44


### Complex elliptope gradient

In [22]:
manifold = ComplexElliptope(n, opt_rank)

@pymanopt.function.Autograd(manifold)
def manifold_cost(Y):
    return np.real(np.trace(M @ Y @ np.conj(Y).T))

problem = pymanopt.Problem(manifold=manifold, cost=manifold_cost)
solver = pymanopt.solvers.SteepestDescent(minstepsize=1e-9)
solution = solver.solve(problem)

Optimizing...
Iteration    Cost                       Gradient norm     
---------    -----------------------    --------------    
   1         +1.3551293986124184e+05    5.49743191e+04    
Terminated - min stepsize reached after 1 iterations, 0.02 seconds.



### Hyperplane rounding

In [14]:
qp_cost = lambda u: -cp.real(cp.quad_form(u, M)).value

In [15]:
complex_hyperplane_rounding(decompose_psd(U.value), qp_cost)

(array([[ 0.08519919-0.09548121j],
        [ 0.06305655+0.17592642j],
        [-0.0416951 +0.02836427j],
        [-0.01172078-0.08780442j],
        [ 0.0865462 -0.04801776j],
        [-0.08931889+0.23751578j],
        [ 0.07614338-0.08094399j],
        [ 0.09554681+0.00865453j],
        [ 0.12409845+0.00348004j],
        [ 0.00324088-0.01744834j],
        [ 0.04323691-0.12139102j],
        [-0.04244527+0.14978684j],
        [ 0.16660813-0.07525077j],
        [-0.0819062 -0.01005515j],
        [-0.12543885+0.2058115j ],
        [-0.10580387-0.03154271j],
        [-0.18620486-0.02979513j],
        [-0.15416302+0.00947798j],
        [-0.0036893 -0.03838838j],
        [ 0.05981327-0.08125232j],
        [ 0.08260892-0.20849576j],
        [ 0.03984368+0.02198208j],
        [-0.08199705+0.02461033j],
        [-0.01485817+0.03481425j],
        [ 0.07726055+0.16013038j],
        [ 0.0053742 -0.11971186j],
        [-0.18534006-0.0930069j ],
        [-0.03237948+0.0931882j ],
        [ 0.01523685